# Plot comparison between $M_0$ and $T_{hold}$

Plot the comparison to evaluate the time healing of the friction on the gouge patch.

2025.01.05 Kurama Okubo

- 2025.01.28 update with the error bar

In [ ]:
import os
import matplotlib.pyplot as plt
from matplotlib.cm import ScalarMappable
import matplotlib as mpl
from matplotlib import ticker

%matplotlib inline
import numpy as np
import pandas as pd
import datetime
from datetime import timedelta
from tqdm import tqdm
import warnings
import time
import pickle 
import seaborn as sns
import copy

import scipy.io as sio
from scipy.signal import decimate

%load_ext autoreload
%autoreload 2

plt.rcParams["font.family"] = 'Arial'
# plt.rcParams["font.sans-serif"] = "DejaVu Sans, Arial, Helvetica, Lucida Grande, Verdana, Geneva, Lucid, Avant Garde, sans-serif"
plt.rcParams["font.size"] = 12
plt.rcParams["xtick.direction"] = "in"
plt.rcParams["xtick.major.size"] = 4.75
plt.rcParams["xtick.major.width"] = 0.75
plt.rcParams["xtick.minor.size"] = 3
plt.rcParams["xtick.minor.width"] = 0.4
plt.rcParams["xtick.minor.visible"] = True

plt.rcParams["ytick.direction"] = "in"
plt.rcParams["ytick.major.size"] = 4.75
plt.rcParams["ytick.major.width"] = 0.75
plt.rcParams["ytick.minor.size"] = 3
plt.rcParams["ytick.minor.width"] = 0.4
plt.rcParams["ytick.minor.visible"] = True

plt.rcParams["savefig.transparent"] = False #True


In [ ]:
# datadir = "../data/M0vsStressDrop/"
# if not os.path.exists(datadir):
#     os.makedirs(datadir)

figdir = "../figure/M0vsStressDrop/"
if not os.path.exists(figdir):
    os.makedirs(figdir)


In [ ]:
expr_id = 87

# Load stress drop data

In [ ]:
fi_stressdrop = f"../../../Experiments/DetectEvent/data/Macroscopic_stats_FB03_{expr_id:03d}.csv"

In [ ]:
df_stressdrop = pd.read_csv(fi_stressdrop, index_col=0)
df_stressdrop.head()

# Load gouge event catalog

In [ ]:
# read gouge event catalog
gougepatch_id = "G3"
Qinv_quart = 50

Nvalidsensors_thresh = 4

finame = f"../../../GougeEventCatalog/data/gougeeventcatalog__fb03-{expr_id:03d}__{gougepatch_id}__Q{Qinv_quart}.csv"
df_gc = pd.read_csv(finame, index_col=0)
df_gc_selected = df_gc[df_gc["Nvalidsensors"] >= Nvalidsensors_thresh].copy()
df_gc_selected.head()

# Link the gouge event and stress drop

In [ ]:
for eid, row in df_gc_selected.iterrows():
    stickslip_id = row.stickslip_id
    df_gc_selected.loc[eid, "Shearstress_before"] = df_stressdrop.loc[stickslip_id, "macroshearstress_beforeevent[MPa]"]
    df_gc_selected.loc[eid, "Shearstress_after"] = df_stressdrop.loc[stickslip_id, "macroshearstress_afterevent[MPa]"]
    df_gc_selected.loc[eid, "Stressdrop"] = df_stressdrop.loc[stickslip_id, "macroshearstress_drop[MPa]"]
    df_gc_selected.loc[eid, "Slip_mean"] = df_stressdrop.loc[stickslip_id, "slip_mean[mm]"]

df_gc_selected.head()

In [ ]:
def M02Mw(M0):
    return (np.log10(M0) - 9.1) * 2.0 / 3.0 # synchronized with OpenSWPC : moment_magnitude ( m0 )

def Mw2M0(Mw):
    return 10**( 1.5 * Mw + 9.05) # synchronized with OpenSWPC : seismic_moment ( mw )
   

In [ ]:
scatter_mc = sns.color_palette("Set1")
scatter_mc

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(5.5, 5.4))
mainmarkersize=7

df_gc_fore = df_gc_selected[df_gc_selected["event_label"]=="F"]
# df_gc_fore.plot.scatter(x="Shearstress_before", y="Mw", s=60, ax=ax, color="r", marker="o", edgecolor="k", zorder=3, label="Foreshock")
ax.errorbar(df_gc_fore["M0"].values, df_gc_fore["Shearstress_before"].values, xerr=df_gc_fore["M0_std"].values,
           capsize=0, fmt="o", markersize=mainmarkersize, color=scatter_mc[0], lw=1, markeredgecolor = "k", label="Foreshocks", zorder=3)


ax.set_xlabel("$M_0$ [Nm]")
ax.set_ylabel("Shear stress before the mainshock [MPa]")


ax.set_xscale('log')
ax.set_yscale('log')

xlimit_scaling = [5e-3, 7]
ax.set_xlim(xlimit_scaling)
ax.set_ylim([0.645, 0.665])

ax.set_yticks(np.arange(0.645, 0.665, 0.005))
ax.set_yticklabels(np.arange(0.645, 0.665, 0.005))

plt.gca().yaxis.set_major_formatter(ticker.ScalarFormatter())
plt.gca().yaxis.set_minor_formatter(ticker.NullFormatter())

ax.legend(loc=3)


#-----------------------#
#--- plot double axis---#
#-----------------------#

# https://pythonmatplotlibtips.blogspot.com/2018/01/add-second-x-axis-below-first-x-axis-python-matplotlib-pyplot.html
ax2 = ax.twiny()
ax2.set_xlabel("$M_w$")
ax2.set_xlim([M02Mw(xlimit_scaling[0]), M02Mw(xlimit_scaling[1])]) # synchronize with the first axis

# major tick
newlabel = np.array([-7.5, -7, -6.5, -6.0,])
ax2.minorticks_on()
ax2.set_xticks(newlabel)

ax.grid(True, c=np.array([230, 230, 230])/255, lw=0.25, zorder=-1)
ax.set_axisbelow('True')

plt.tight_layout()

plt.savefig(figdir+f"/M0vsSSbefore_fb03-{expr_id:03d}_{gougepatch_id}_Q{Qinv_quart}.png", format="png", dpi=200)
# plt.savefig(figdir+f"/M0vsSSbefore_fb03-{expr_id:03d}_{gougepatch_id}_Q{Qinv_quart}.pdf")


In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(5.5, 5.4))
mainmarkersize=7

df_gc_fore = df_gc_selected[df_gc_selected["event_label"]=="F"]
# df_gc_fore.plot.scatter(x="Shearstress_before", y="Mw", s=60, ax=ax, color="r", marker="o", edgecolor="k", zorder=3, label="Foreshock")
ax.errorbar(df_gc_fore["M0"].values, df_gc_fore["Stressdrop"].values, xerr=df_gc_fore["M0_std"].values,
           capsize=0, fmt="o", markersize=mainmarkersize, color=scatter_mc[0], lw=1, markeredgecolor = "k", label="Foreshocks", zorder=3)


ax.set_xlabel("$M_0$ [Nm]")
ax.set_ylabel("Macroscopic Δτ [MPa]")

ax.set_xscale('log')
ax.set_yscale('log')

xlimit_scaling = [5e-3, 7]
ax.set_xlim(xlimit_scaling)
ax.set_ylim([1.5e-2, 3.0e-2])

ax.set_yticks(np.arange(0.02, 0.04, 0.01))
ax.set_yticklabels(np.arange(0.02, 0.04, 0.01))

plt.gca().yaxis.set_major_formatter(ticker.ScalarFormatter())
plt.gca().yaxis.set_minor_formatter(ticker.NullFormatter())

ax.legend(loc=3)


#-----------------------#
#--- plot double axis---#
#-----------------------#

# https://pythonmatplotlibtips.blogspot.com/2018/01/add-second-x-axis-below-first-x-axis-python-matplotlib-pyplot.html
ax2 = ax.twiny()
ax2.set_xlabel("$M_w$")
ax2.set_xlim([M02Mw(xlimit_scaling[0]), M02Mw(xlimit_scaling[1])]) # synchronize with the first axis

# major tick
newlabel = np.array([-7.5, -7, -6.5, -6.0,])
ax2.minorticks_on()
ax2.set_xticks(newlabel)

ax.grid(True, c=np.array([230, 230, 230])/255, lw=0.25, zorder=-1)
ax.set_axisbelow('True')

plt.tight_layout()

plt.savefig(figdir+f"/M0vsStressdrop_fb03-{expr_id:03d}_{gougepatch_id}_Q{Qinv_quart}.png", format="png", dpi=200)
# plt.savefig(figdir+f"/M0vsStressdrop_fb03-{expr_id:03d}_{gougepatch_id}_Q{Qinv_quart}.pdf")


In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(5.5, 5.4))
mainmarkersize=7

df_gc_fore = df_gc_selected[df_gc_selected["event_label"]=="F"]
# df_gc_fore.plot.scatter(x="Shearstress_before", y="Mw", s=60, ax=ax, color="r", marker="o", edgecolor="k", zorder=3, label="Foreshock")
ax.errorbar(df_gc_fore["M0"].values, df_gc_fore["Slip_mean"].values*1e3, xerr=df_gc_fore["M0_std"].values,
           capsize=0, fmt="o", markersize=mainmarkersize, color=scatter_mc[0], lw=1, markeredgecolor = "k", label="Foreshocks", zorder=3)


ax.set_xlabel("$M_0$ [Nm]")
# ax.set_ylabel("Macroscopic Δτ [MPa]")
ax.set_ylabel("Mean slip of main shock [mm]")

ax.set_xscale('log')
ax.set_yscale('log')

xlimit_scaling = [5e-3, 7]
ax.set_xlim(xlimit_scaling)
ax.set_ylim([25, 50])

ax.set_yticks(np.arange(30, 60, 10))
ax.set_yticklabels(np.arange(30, 60, 10))

# plt.gca().yaxis.set_major_formatter(ticker.ScalarFormatter())
# plt.gca().yaxis.set_minor_formatter(ticker.NullFormatter())

ax.legend(loc=3)


#-----------------------#
#--- plot double axis---#
#-----------------------#

# https://pythonmatplotlibtips.blogspot.com/2018/01/add-second-x-axis-below-first-x-axis-python-matplotlib-pyplot.html
ax2 = ax.twiny()
ax2.set_xlabel("$M_w$")
ax2.set_xlim([M02Mw(xlimit_scaling[0]), M02Mw(xlimit_scaling[1])]) # synchronize with the first axis

# major tick
newlabel = np.array([-7.5, -7, -6.5, -6.0,])
ax2.minorticks_on()
ax2.set_xticks(newlabel)

ax.grid(True, c=np.array([230, 230, 230])/255, lw=0.25, zorder=-1)
ax.set_axisbelow('True')

plt.tight_layout()

plt.savefig(figdir+f"/M0vsMeanSlip_fb03-{expr_id:03d}_{gougepatch_id}_Q{Qinv_quart}.png", format="png", dpi=200)
# plt.savefig(figdir+f"/M0vsMeanSlip_fb03-{expr_id:03d}_{gougepatch_id}_Q{Qinv_quart}.pdf")
